In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os

# Ruta a la carpeta donde están los labels de los serrados
carpeta_labels_serrado = '/content/drive/MyDrive/TFG/Anotaciones/Serrados'

# Recorremos todos los archivos .txt de esa carpeta
for archivo in os.listdir(carpeta_labels_serrado):
    if archivo.endswith('.txt'):
        ruta = os.path.join(carpeta_labels_serrado, archivo)
        nuevas_lineas = []

        with open(ruta, 'r') as f:
            for linea in f:
                partes = linea.strip().split()
                if len(partes) > 0:
                    partes[0] = '1'  # Reemplazamos la clase 0 por 1, que es la clase de serrado
                    nuevas_lineas.append(' '.join(partes))

        # Sobrescribimos el archivo con la clase corregida
        with open(ruta, 'w') as f:
            f.write('\n'.join(nuevas_lineas))

print("✅ Clases de los archivos de 'Serrado' cambiadas a 1 correctamente.")


✅ Clases de los archivos de 'Serrado' cambiadas a 1 correctamente.


In [ ]:
import os
import random
import shutil
import cv2
import albumentations as A

# Rutas de origen
NORMAL_IMG = '/content/drive/MyDrive/TFG/colonoscopias/Normal'
ADENOMA_IMG = '/content/drive/MyDrive/TFG/colonoscopias/Adenomas'
ADENOMA_LBL = '/content/drive/MyDrive/TFG/Anotaciones/Adenomas'
SERRADO_IMG = '/content/drive/MyDrive/TFG/colonoscopias/Serrados'
SERRADO_LBL = '/content/drive/MyDrive/TFG/Anotaciones/Serrados'

# Carpeta final para YOLO (detección)
DATASET_BASE = '/content/drive/MyDrive/TFG/dataset_deteccion'
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(DATASET_BASE, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(DATASET_BASE, 'labels', split), exist_ok=True)

# Ratios de división
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1


In [ ]:

# Función para renombrar extensiones .JPG a .jpg
def rename_jpg(dir_path):
    for f in os.listdir(dir_path):
        old_path = os.path.join(dir_path, f)
        if not os.path.isfile(old_path):
            continue
        name, ext = os.path.splitext(f)
        ext_lower = ext.lower()
        if ext != ext_lower:
            new_name = name + ext_lower
            new_path = os.path.join(dir_path, new_name)
            os.rename(old_path, new_path)

# Renombrar imágenes en todas las carpetas
rename_jpg(NORMAL_IMG)
rename_jpg(ADENOMA_IMG)
rename_jpg(SERRADO_IMG)

# Función para listar imágenes y etiquetas
def list_imgs_and_labels(img_dir, lbl_dir=None, class_id=None):
    img_files = [f for f in os.listdir(img_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    data_list = []

    for img in img_files:
        base = os.path.splitext(img)[0]
        img_path = os.path.join(img_dir, img)
        lbl_path = os.path.join(lbl_dir, base + '.txt') if lbl_dir else None

        if lbl_dir and not os.path.exists(lbl_path):
            continue  # Omitir imágenes de Adenomas y Serrados sin etiquetas

        if lbl_dir:
            data_list.append((img_path, lbl_path))
        else:
            # Para imágenes "Normal", creamos un txt vacío
            data_list.append((img_path, None))

    return data_list

# Listar imágenes y etiquetas
norm_data = list_imgs_and_labels(NORMAL_IMG)  # Sin etiquetas
ade_data = list_imgs_and_labels(ADENOMA_IMG, ADENOMA_LBL)
ser_data = list_imgs_and_labels(SERRADO_IMG, SERRADO_LBL)

random.shuffle(norm_data)
random.shuffle(ade_data)
random.shuffle(ser_data)

# Función para dividir los datos
def split_data(data_list, tr_ratio, val_ratio):
    n_total = len(data_list)
    n_train = int(n_total * tr_ratio)
    n_val = int(n_total * val_ratio)
    n_test = n_total - n_train - n_val
    return data_list[:n_train], data_list[n_train:n_train + n_val], data_list[n_train + n_val:]

# Dividir los datos en train/val/test
norm_tr, norm_val, norm_test = split_data(norm_data, train_ratio, val_ratio)
ade_tr, ade_val, ade_test = split_data(ade_data, train_ratio, val_ratio)
ser_tr, ser_val, ser_test = split_data(ser_data, train_ratio, val_ratio)


In [ ]:
import albumentations as A

# Definición del pipeline completo de Data Augmentation
augment_transform = A.Compose(
    [
        # Aplica un volteo horizontal aleatorio con probabilidad del 50%
        A.HorizontalFlip(p=0.5),

        # Ajusta aleatoriamente el brillo y el contraste (±20%), con probabilidad 50%
        A.RandomBrightnessContrast(
            brightness_limit=0.2,
            contrast_limit=0.2,
            p=0.5
        ),

        # Rota la imagen aleatoriamente en un rango de -15 a +15 grados, con probabilidad 50%
        A.Rotate(limit=15, p=0.5),

        # Modifica ligeramente el tono (hue), saturación y valor (brillo) de colores con una probabilidad de 30%
        A.HueSaturationValue(
            hue_shift_limit=20,   # Variación máxima en el tono (color)
            sat_shift_limit=30,   # Variación máxima en saturación (intensidad de color)
            val_shift_limit=20,   # Variación máxima en valor (brillo)
            p=0.3
        ),

        # Aplica traslaciones, escalados o rotaciones ligeras adicionales con probabilidad 50%
        A.ShiftScaleRotate(
            shift_limit=0.1,      # Traslada imagen hasta ±10% en ambas direcciones (x, y)
            scale_limit=0.1,      # Escala la imagen hasta ±10% (más pequeña o más grande)
            rotate_limit=0,       # No rota más aquí porque ya está en Rotate arriba
            border_mode=0,        # Rellena bordes nuevos con negro (valor 0)
            p=0.5
        ),

        # Aplica distorsión (cizallamiento) horizontal o vertical hasta ±10 grados, con probabilidad 30%
        A.Affine(
            shear=(-10, 10),
            p=0.3
        ),

        # Elimina aleatoriamente pequeñas regiones rectangulares (dropout),
        # generando robustez ante posibles oclusiones, con probabilidad 20%
        A.CoarseDropout(
            max_holes=4,          # Hasta 4 regiones pueden ser eliminadas
            max_height=16,        # Altura máxima de cada región eliminada
            max_width=16,         # Ancho máximo de cada región eliminada
            fill_value=0,         # Valor con el que se rellenan las regiones eliminadas (negro)
            p=0.2
        ),

        # Nota importante:
        # Mosaic Augmentation NO se puede realizar directamente con Albumentations.
        # En YOLOv8 (Ultralytics) se configura directamente desde los parámetros de entrenamiento.
    ],

    # El siguiente bloque es para asegurar que las bounding boxes
    # (regiones etiquetadas como pólipos) también sean transformadas
    # adecuadamente junto con cada imagen.
    bbox_params=A.BboxParams(format='yolo', label_fields=['labels'], min_area=1, min_visibility=0.1)
)


<ipython-input-9-98d7791bb66e>:44: UserWarning: Argument(s) 'max_holes, max_height, max_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(


In [ ]:
import os
import cv2
import shutil

def copy_and_augment(data_list, subset, class_id=None):
    img_dst = os.path.join(DATASET_BASE, 'images', subset)
    lbl_dst = os.path.join(DATASET_BASE, 'labels', subset)

    os.makedirs(img_dst, exist_ok=True)
    os.makedirs(lbl_dst, exist_ok=True)

    n_aug = 3 if subset == 'train' else 1  # 3 aumentos solo para train

    for img_path, lbl_path in data_list:
        base = os.path.splitext(os.path.basename(img_path))[0]

        # Copiar imagen original
        new_img_path = os.path.join(img_dst, base + '.jpg')
        shutil.copy2(img_path, new_img_path)

        if lbl_path:
            # Copiar etiquetas existentes
            new_lbl_path = os.path.join(lbl_dst, base + '.txt')
            shutil.copy2(lbl_path, new_lbl_path)
        else:
            # Crear etiqueta vacía para imágenes "Normal"
            new_lbl_path = os.path.join(lbl_dst, base + '.txt')
            open(new_lbl_path, 'w').close()

        # Augment solo en train y solo para Adenoma/Serrado (con labels)
        if subset != 'train' or lbl_path is None:
            continue

        img_bgr = cv2.imread(img_path)
        if img_bgr is None:
            continue

        h, w = img_bgr.shape[:2]
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

        # Leer bboxes directamente en YOLO (no convertir manualmente a píxeles)
        bboxes_list = []
        labels_list = []
        with open(lbl_path, 'r') as f_lbl:
            for line in f_lbl:
                parts = line.strip().split()
                cls_id = int(parts[0])
                x_center, y_center, width, height = map(float, parts[1:])
                bboxes_list.append([x_center, y_center, width, height])  # YOLO normalizado directo
                labels_list.append(cls_id)

        # Generar imágenes aumentadas usando Albumentations directamente
        for i in range(n_aug):
            aug_result = augment_transform(image=img_rgb, bboxes=bboxes_list, labels=labels_list)
            aug_img = cv2.cvtColor(aug_result['image'], cv2.COLOR_RGB2BGR)

            aug_img_path = os.path.join(img_dst, f"{base}_aug{i}.jpg")
            cv2.imwrite(aug_img_path, aug_img)

            aug_lbl_path = os.path.join(lbl_dst, f"{base}_aug{i}.txt")

            # Guardar etiquetas transformadas directamente
            with open(aug_lbl_path, 'w') as f_out:
                for bbox, cls_id in zip(aug_result['bboxes'], aug_result['labels']):
                    x_center, y_center, width, height = bbox  # Ya formato YOLO normalizado
                    f_out.write(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")


In [ ]:
# 4. Copiar y augmentar para Adenoma y Serrado

copy_and_augment(norm_tr, 'train')
copy_and_augment(ade_tr, 'train', 1)
copy_and_augment(ser_tr, 'train', 2)

print("\n🎉 Dataset completado.")


🎉 Dataset completado.


In [ ]:
# Copiar explícitamente archivos para validación (val), sin aumentación
copy_and_augment(norm_val, 'val')
copy_and_augment(ade_val, 'val', 1)
copy_and_augment(ser_val, 'val', 2)

# Copiar explícitamente archivos para test, sin aumentación
copy_and_augment(norm_test, 'test')
copy_and_augment(ade_test, 'test', 1)
copy_and_augment(ser_test, 'test', 2)
